In [2]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
import numpy as np
import time
import config as cfg

begin = time.strftime('%Y-%m-%d_%H-%M-%S')

LOGDIR = './save'

tf.logging.set_verbosity(tf.logging.ERROR)

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

#start_learning_rate = 0.5e-3  # 1e-3
start_learning_rate = 0.5e-2  # 1e-3
#adjust_learning_rate = 1e-5
adjust_learning_rate = 1e-3

#onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=4)
onehot_labels = tf.one_hot(indices = tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth = cfg.NUM_KEYS)

loss = tf.losses.softmax_cross_entropy( onehot_labels = onehot_labels, logits = model.y)
# loss = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

loss_val = tf.losses.softmax_cross_entropy( onehot_labels = onehot_labels, logits = model.y)
# loss_val = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
# train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
# tf.scalar_summary("loss", loss)
tf.summary.scalar("loss", loss)

tf.summary.scalar("loss_val", loss_val)

# merge all summaries into a single op
# merged_summary_op = tf.merge_all_summaries()
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)
# saver.restore(sess, "save/model.ckpt")

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

#epochs = 30   
epochs = 10   
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        # print('Each item in xs : shape is...')
        # print(xs[0].shape)
        
        # train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.7})
        loss_value = loss.eval(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0})
        
        # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))
        print("Epoch: %d, Step: %d, Loss: %g" % (epoch, i, loss_value))

        if i % 10 == 0:
            xs_val, ys_val = driving_data.LoadValBatch(batch_size)
            # xs, ys = driving_data.LoadTrainBatch(batch_size)
            loss_val = loss.eval(feed_dict={model.x:xs_val, model.y_: ys_val, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss_val: %g" % (epoch, i, loss_val))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
            print("Model saved in file: %s" % filename)


correct_prediction = tf.equal(tf.argmax(onehot_labels, 1), tf.argmax(model.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Train Accuracy:', sess.run(accuracy, feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}))
print('Validation Accuracy:', sess.run(accuracy, feed_dict={model.x: xs_val, model.y_: ys_val, model.keep_prob: 1.0}))

end = time.strftime('%Y-%m-%d_%H-%M-%S')
print('begin: ', begin)
print('end: ', end)

print("Run the command line:\n",
      "--> tensorboard --logdir=./logs --port=6006",
      "\nThen open http://0.0.0.0:6006/ into your web browser")

# os.system("sudo -s shutdown -h now")


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch: 0, Step: 0, Loss: 1.66453
Epoch: 0, Step: 0, Loss_val: 1.84422
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 1, Loss: 1.77438
Epoch: 0, Step: 2, Loss: 1.76669
Epoch: 0, Step: 3, Loss: 1.71347
Epoch: 0, Step: 4, Loss: 1.66868
Epoch: 0, Step: 5, Loss: 1.58392
Epoch: 0, Step: 6, Loss: 1.63907
Epoch: 0, Step: 7, Loss: 1.63244
Epoch: 0, Step: 8, Loss: 1.60309
Epoch: 0, Step: 9, Loss: 1.58424
Epoch: 0, Step: 10, Loss: 1.6137
Epoch: 0, Step: 10, Loss_val: 1.59669
Epoch: 0, Step: 11, Loss: 1.55426
Epoch: 0, Step: 12, Loss: 1.61094
Epoch: 0, Step: 13, Loss: 1.57424
Epoch: 0, Step: 14, Loss: 1.5586
Epoch: 0, Step: 15, Loss: 1.49263
Epoch: 0, Step: 16, Loss: 1.51554
Epoch: 0, Step: 17, Loss: 1.52419
Epoch: 0, Step: 18, Loss: 1.51017
Epoch: 0, Step: 19, Loss: 1.53049
Epoch: 0, Step: 20, Loss: 1.50171
Epoch: 0, Step: 20, Loss_val: 1.52782
Epoch: 0, Step: 21, Loss: 1.49347
Epoch: 0, Step: 22, Loss: 1.37057
Epoch: 0, Step: 23, Loss: 1.35734
Epoch: 0, Step: 24, Loss: 1.27546
Epoch: 0, 

Epoch: 0, Step: 206, Loss: 0.243887
Epoch: 0, Step: 207, Loss: 0.295622
Epoch: 0, Step: 208, Loss: 0.50114
Epoch: 0, Step: 209, Loss: 0.305558
Epoch: 0, Step: 210, Loss: 0.31063
Epoch: 0, Step: 210, Loss_val: 0.270594
Epoch: 0, Step: 211, Loss: 0.353629
Epoch: 0, Step: 212, Loss: 0.278194
Epoch: 0, Step: 213, Loss: 0.415366
Epoch: 0, Step: 214, Loss: 0.396482
Epoch: 0, Step: 215, Loss: 0.297295
Epoch: 0, Step: 216, Loss: 0.325409
Epoch: 0, Step: 217, Loss: 0.430107
Epoch: 0, Step: 218, Loss: 0.311827
Epoch: 0, Step: 219, Loss: 0.357371
Epoch: 0, Step: 220, Loss: 0.441448
Epoch: 0, Step: 220, Loss_val: 0.359305
Epoch: 0, Step: 221, Loss: 0.308748
Epoch: 0, Step: 222, Loss: 0.259356
Epoch: 0, Step: 223, Loss: 0.377913
Epoch: 0, Step: 224, Loss: 0.302347
Epoch: 0, Step: 225, Loss: 0.41801
Epoch: 0, Step: 226, Loss: 0.398582
Epoch: 0, Step: 227, Loss: 0.257073
Epoch: 0, Step: 228, Loss: 0.344081
Epoch: 0, Step: 229, Loss: 0.317498
Epoch: 0, Step: 230, Loss: 0.393167
Epoch: 0, Step: 230, Lo

Epoch: 1, Step: 134, Loss: 0.36956
Epoch: 1, Step: 135, Loss: 0.36224
Epoch: 1, Step: 136, Loss: 0.488154
Epoch: 1, Step: 137, Loss: 0.247982
Epoch: 1, Step: 138, Loss: 0.304601
Epoch: 1, Step: 139, Loss: 0.365662
Epoch: 1, Step: 140, Loss: 0.402193
Epoch: 1, Step: 140, Loss_val: 0.390555
Epoch: 1, Step: 141, Loss: 0.292492
Epoch: 1, Step: 142, Loss: 0.277834
Epoch: 1, Step: 143, Loss: 0.321172
Epoch: 1, Step: 144, Loss: 0.306124
Epoch: 1, Step: 145, Loss: 0.339702
Epoch: 1, Step: 146, Loss: 0.323642
Epoch: 1, Step: 147, Loss: 0.346136
Epoch: 1, Step: 148, Loss: 0.361141
Epoch: 1, Step: 149, Loss: 0.232352
Epoch: 1, Step: 150, Loss: 0.180849
Epoch: 1, Step: 150, Loss_val: 0.288509
Epoch: 1, Step: 151, Loss: 0.23654
Epoch: 1, Step: 152, Loss: 0.332885
Epoch: 1, Step: 153, Loss: 0.314603
Epoch: 1, Step: 154, Loss: 0.277106
Epoch: 1, Step: 155, Loss: 0.341049
Epoch: 1, Step: 156, Loss: 0.198754
Epoch: 1, Step: 157, Loss: 0.320566
Epoch: 1, Step: 158, Loss: 0.315637
Epoch: 1, Step: 159, Lo

Epoch: 2, Step: 62, Loss: 0.315632
Epoch: 2, Step: 63, Loss: 0.199893
Epoch: 2, Step: 64, Loss: 0.190147
Epoch: 2, Step: 65, Loss: 0.286751
Epoch: 2, Step: 66, Loss: 0.314852
Epoch: 2, Step: 67, Loss: 0.271561
Epoch: 2, Step: 68, Loss: 0.329847
Epoch: 2, Step: 69, Loss: 0.289801
Epoch: 2, Step: 70, Loss: 0.302778
Epoch: 2, Step: 70, Loss_val: 0.310532
Epoch: 2, Step: 71, Loss: 0.305326
Epoch: 2, Step: 72, Loss: 0.306004
Epoch: 2, Step: 73, Loss: 0.262204
Epoch: 2, Step: 74, Loss: 0.220729
Epoch: 2, Step: 75, Loss: 0.265124
Epoch: 2, Step: 76, Loss: 0.335743
Epoch: 2, Step: 77, Loss: 0.224258
Epoch: 2, Step: 78, Loss: 0.262196
Epoch: 2, Step: 79, Loss: 0.348891
Epoch: 2, Step: 80, Loss: 0.34443
Epoch: 2, Step: 80, Loss_val: 0.211624
Epoch: 2, Step: 81, Loss: 0.413192
Epoch: 2, Step: 82, Loss: 0.250259
Epoch: 2, Step: 83, Loss: 0.242332
Epoch: 2, Step: 84, Loss: 0.355945
Epoch: 2, Step: 85, Loss: 0.341368
Epoch: 2, Step: 86, Loss: 0.224357
Epoch: 2, Step: 87, Loss: 0.233713
Epoch: 2, Ste

...

Epoch: 6, Step: 188, Loss: 0.107928
Epoch: 6, Step: 189, Loss: 0.116463
Epoch: 6, Step: 190, Loss: 0.157574
Epoch: 6, Step: 190, Loss_val: 0.266509
Epoch: 6, Step: 191, Loss: 0.154122
Epoch: 6, Step: 192, Loss: 0.146722
Epoch: 6, Step: 193, Loss: 0.191441
Epoch: 6, Step: 194, Loss: 0.129188
Epoch: 6, Step: 195, Loss: 0.202045
Epoch: 6, Step: 196, Loss: 0.165389
Epoch: 6, Step: 197, Loss: 0.13943
Epoch: 6, Step: 198, Loss: 0.173169
Epoch: 6, Step: 199, Loss: 0.115035
Epoch: 6, Step: 200, Loss: 0.14998
Epoch: 6, Step: 200, Loss_val: 0.211655
Model saved in file: ./save/model.ckpt
Epoch: 6, Step: 201, Loss: 0.154097
Epoch: 6, Step: 202, Loss: 0.182515
Epoch: 6, Step: 203, Loss: 0.131682
Epoch: 6, Step: 204, Loss: 0.193688
Epoch: 6, Step: 205, Loss: 0.126207
Epoch: 6, Step: 206, Loss: 0.132885
Epoch: 6, Step: 207, Loss: 0.178585
Epoch: 6, Step: 208, Loss: 0.175631
Epoch: 6, Step: 209, Loss: 0.0685477
Epoch: 6, Step: 210, Loss: 0.119515
Epoch: 6, Step: 210, Loss_val: 0.162828
Epoch: 6, Step

Epoch: 7, Step: 116, Loss: 0.110374
Epoch: 7, Step: 117, Loss: 0.193702
Epoch: 7, Step: 118, Loss: 0.152062
Epoch: 7, Step: 119, Loss: 0.124577
Epoch: 7, Step: 120, Loss: 0.135941
Epoch: 7, Step: 120, Loss_val: 0.42078
Epoch: 7, Step: 121, Loss: 0.0762729
Epoch: 7, Step: 122, Loss: 0.161084
Epoch: 7, Step: 123, Loss: 0.205604
Epoch: 7, Step: 124, Loss: 0.16937
Epoch: 7, Step: 125, Loss: 0.183118
Epoch: 7, Step: 126, Loss: 0.178882
Epoch: 7, Step: 127, Loss: 0.165442
Epoch: 7, Step: 128, Loss: 0.0790702
Epoch: 7, Step: 129, Loss: 0.12094
Epoch: 7, Step: 130, Loss: 0.196709
Epoch: 7, Step: 130, Loss_val: 0.283339
Epoch: 7, Step: 131, Loss: 0.13074
Epoch: 7, Step: 132, Loss: 0.134402
Epoch: 7, Step: 133, Loss: 0.0796836
Epoch: 7, Step: 134, Loss: 0.167225
Epoch: 7, Step: 135, Loss: 0.0913536
Epoch: 7, Step: 136, Loss: 0.160421
Epoch: 7, Step: 137, Loss: 0.114714
Epoch: 7, Step: 138, Loss: 0.102749
Epoch: 7, Step: 139, Loss: 0.130608
Epoch: 7, Step: 140, Loss: 0.18764
Epoch: 7, Step: 140, 

Epoch: 8, Step: 42, Loss: 0.132632
Epoch: 8, Step: 43, Loss: 0.132344
Epoch: 8, Step: 44, Loss: 0.0894461
Epoch: 8, Step: 45, Loss: 0.195875
Epoch: 8, Step: 46, Loss: 0.0831616
Epoch: 8, Step: 47, Loss: 0.111624
Epoch: 8, Step: 48, Loss: 0.0487109
Epoch: 8, Step: 49, Loss: 0.131321
Epoch: 8, Step: 50, Loss: 0.0891634
Epoch: 8, Step: 50, Loss_val: 0.284388
Epoch: 8, Step: 51, Loss: 0.157785
Epoch: 8, Step: 52, Loss: 0.173999
Epoch: 8, Step: 53, Loss: 0.177221
Epoch: 8, Step: 54, Loss: 0.113126
Epoch: 8, Step: 55, Loss: 0.144162
Epoch: 8, Step: 56, Loss: 0.117342
Epoch: 8, Step: 57, Loss: 0.135786
Epoch: 8, Step: 58, Loss: 0.157006
Epoch: 8, Step: 59, Loss: 0.141684
Epoch: 8, Step: 60, Loss: 0.125879
Epoch: 8, Step: 60, Loss_val: 0.209892
Epoch: 8, Step: 61, Loss: 0.100342
Epoch: 8, Step: 62, Loss: 0.172278
Epoch: 8, Step: 63, Loss: 0.14158
Epoch: 8, Step: 64, Loss: 0.105419
Epoch: 8, Step: 65, Loss: 0.115603
Epoch: 8, Step: 66, Loss: 0.140521
Epoch: 8, Step: 67, Loss: 0.0901354
Epoch: 8

Epoch: 8, Step: 248, Loss: 0.120936
Epoch: 8, Step: 249, Loss: 0.0496254
Epoch: 8, Step: 250, Loss: 0.176861
Epoch: 8, Step: 250, Loss_val: 0.151665
Epoch: 8, Step: 251, Loss: 0.135893
Epoch: 8, Step: 252, Loss: 0.0636316
Epoch: 8, Step: 253, Loss: 0.106109
Epoch: 8, Step: 254, Loss: 0.10435
Epoch: 8, Step: 255, Loss: 0.0931108
Epoch: 8, Step: 256, Loss: 0.230655
Epoch: 8, Step: 257, Loss: 0.115618
Epoch: 8, Step: 258, Loss: 0.0788011
Epoch: 8, Step: 259, Loss: 0.0541098
Epoch: 8, Step: 260, Loss: 0.184645
Epoch: 8, Step: 260, Loss_val: 0.140448
Epoch: 8, Step: 261, Loss: 0.0949977
Epoch: 8, Step: 262, Loss: 0.106893
Epoch: 8, Step: 263, Loss: 0.0605949
Epoch: 8, Step: 264, Loss: 0.15396
Epoch: 8, Step: 265, Loss: 0.0272111
Epoch: 8, Step: 266, Loss: 0.0955432
Epoch: 8, Step: 267, Loss: 0.0992929
Epoch: 8, Step: 268, Loss: 0.202777
Epoch: 8, Step: 269, Loss: 0.0922825
Epoch: 8, Step: 270, Loss: 0.0802845
Epoch: 8, Step: 270, Loss_val: 0.0739778
Epoch: 8, Step: 271, Loss: 0.0742251
Epoc

Epoch: 9, Step: 176, Loss: 0.131244
Epoch: 9, Step: 177, Loss: 0.0797963
Epoch: 9, Step: 178, Loss: 0.0645799
Epoch: 9, Step: 179, Loss: 0.120062
Epoch: 9, Step: 180, Loss: 0.133674
Epoch: 9, Step: 180, Loss_val: 0.227508
Epoch: 9, Step: 181, Loss: 0.132345
Epoch: 9, Step: 182, Loss: 0.131399
Epoch: 9, Step: 183, Loss: 0.157543
Epoch: 9, Step: 184, Loss: 0.11432
Epoch: 9, Step: 185, Loss: 0.110587
Epoch: 9, Step: 186, Loss: 0.0854928
Epoch: 9, Step: 187, Loss: 0.0913179
Epoch: 9, Step: 188, Loss: 0.107638
Epoch: 9, Step: 189, Loss: 0.0942293
Epoch: 9, Step: 190, Loss: 0.0937905
Epoch: 9, Step: 190, Loss_val: 0.0979726
Epoch: 9, Step: 191, Loss: 0.146418
Epoch: 9, Step: 192, Loss: 0.148868
Epoch: 9, Step: 193, Loss: 0.0836132
Epoch: 9, Step: 194, Loss: 0.0612679
Epoch: 9, Step: 195, Loss: 0.123189
Epoch: 9, Step: 196, Loss: 0.120946
Epoch: 9, Step: 197, Loss: 0.120766
Epoch: 9, Step: 198, Loss: 0.0805077
Epoch: 9, Step: 199, Loss: 0.0801972
Epoch: 9, Step: 200, Loss: 0.108804
Epoch: 9, 